最近和一位做量化的私募大佬聊了一下行情，他给我发了这张图片。

<div style='width:30%;text-align:center;margin: 0 auto 1rem'>
<img src='https://images.jieyu.ai/images/2024/10/roger-trend.jpg'>
<span style='font-size:0.6rem'></span>
</div>

这个底部点位，他**又一次**精准命中了（3143那个点，不是3066。周五上证实际下探到3152点）。不过，我更好奇的是他的研究方法，也就图的下半部分。知道大致的底之后，再结合缺口、前低等一些信息，确实有可能比较精准地预测底部点位。

通过akshare获取指定日期间的行情数据。

我当时就回了一句，最近忙着上课，等有时间了，把这个三角形检测写出来。

这个检测并不难，写一个教学示例，一个小时的时间足够了。

在分享我的算法之前，先推荐一个外网的[方案](https://www.youtube.com/watch?v=b5m7BZAHysk)。同样是教学代码，显然不如我随手写的优雅，先小小自得一下。不过，这样的好处就是，他的代码可能更容易读懂。

所谓旗形整理（或者说三角形检测），就是下面这张图：


<div style='width:50%;text-align:center;margin: 0 auto 1rem'>
<img src='https://images.jieyu.ai/images/2024/10/flag-pattern-1.jpg'>
<span style='font-size:0.6rem'></span>
</div>



在这张图，每次上涨的局部高点连接起来，构成压力线；而下跌的局部低点连起来，构成支撑线。

如果我们再在开始的位置画一条竖线，就构成了一个小旗帜，这就是旗形的来由。

旗形整理的特别之处是，整理何时结束似乎是可以预测的，因为这两条线之间的交易空间会越来越窄。

**当它小于一个ATR时**，就是整理必须结束，即将选择方向的时候。

下图显示了随时间推移，震荡幅度越来越小的情况。

<div style='width:50%;text-align:center;margin: 0 auto 1rem'>
<img src='https://images.jieyu.ai/images/2024/10/flag-pattern-2.jpg'>
<span style='font-size:0.6rem'></span>
</div>


最终，股价会选择方向。一旦选择方向，就往往会有一波较大的行情（或者下跌）：

<div style='width:50%;text-align:center;margin: 0 auto 1rem'>
<img src='https://images.jieyu.ai/images/2024/10/flag-pattern-3.jpg'>
<span style='font-size:0.6rem'></span>
</div>


所以，能够自动化检测旗形整理，有以下作用：



1. 如果当前处理在旗形整理中，可以设定合理的波段期望。
2. 检测临近整理结束，可以减仓等待方向。
3. 一旦方向确定，立即加仓。

现在，我们就来看如何实现。首先，我们有这样一个标的：

<div style='width:50%;text-align:center;margin: 0 auto 1rem'>
<img src='https://images.jieyu.ai/images/2024/10/605158-1.png'>
<span style='font-size:0.6rem'></span>
</div>


这是已经上涨后的。我们再来看它上涨前的：

<div style='width:50%;text-align:center;margin: 0 auto 1rem'>
<img src='https://images.jieyu.ai/images/2024/10/605158.png'>
<span style='font-size:0.6rem'></span>
</div>


肉眼来看，一个旗形整理似有若无。

我们的算法分这样几步：

1. 找到每阶段的峰和谷的坐标
2. 通过这些坐标及它们的收盘价，进行趋势线拟合
3. 通过np.poly1d生成趋势线
4. 将趋势线和k线图画在一张图上

首先，我们来获取相关数据。


In [1]:
import akshare as ak
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go

code = "sh605158"

def get_bars(symbol, start, end):
    start_ = f"{start.year:04d}{start.month:02d}{start.day:02d}"
    end_ = f"{end.year:04d}{end.month:02d}{end.day:02d}"
    bars = ak.stock_zh_a_daily(symbol=code, start_date=start_, end_date = end_, adjust="qfq")

    return bars

df = get_bars(code, datetime.date(2024,1,1), datetime.date(2024,10,18))
df.tail()

,date,open,high,low,close,volume,amount,outstanding_share,turnover
185,2024-10-14,8.53,9.00,8.36,9.00,13635627.0,120173201.0,511420000.0,0.026662
186,2024-10-15,9.05,9.56,9.05,9.11,22525713.0,209376045.0,511420000.0,0.044045
187,2024-10-16,9.05,9.54,8.89,9.22,14126252.0,129985106.0,511420000.0,0.027622
188,2024-10-17,9.27,9.49,9.08,9.39,13667642.0,127023312.0,511420000.0,0.026725
189,2024-10-18,9.32,9.95,9.27,9.88,14676130.0,141660370.0,511420000.0,0.028697


绘制未上涨前整理期图形

In [7]:
RED = "#FF4136"
GREEN = "#3DAA70"

def candlestick(df):
    candle = go.Candlestick(x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'],
                    line=dict({"width": 1}),
                    name="K 线",
                    increasing = {
                        "fillcolor":"rgba(255,255,255,0.9)",
                        "line": dict({"color": RED})
                    },
                    decreasing = {
                        "fillcolor": GREEN, 
                        "line": dict(color =  GREEN)
                    })

    fig = go.Figure(data=[candle])

    fig.show()

candlestick(df.iloc[:179])

## 定义算法

In [13]:
def find_runs(x):
    x = np.asanyarray(x)
    n = len(x)

    loc_run_start = np.empty(n, dtype=bool)
    loc_run_start[0] = True
    np.not_equal(x[:-1], x[1:], out=loc_run_start[1:])
    run_starts = np.nonzero(loc_run_start)[0]

    # find run values
    run_values = x[loc_run_start]

    # find run lengths
    run_lengths = np.diff(np.append(run_starts, n))

    return run_values, run_starts, run_lengths

def find_peak_pivots(df, win):
    local_high = df.close.rolling(win).apply(lambda x: x.argmax()== win-1)
    local_high[:win] = 0
    
    v,s,l = find_runs(local_high)

    peaks = []
    i = 0
    while i < len(v):
        if l[i] >= win // 2:
            if s[i] > 0:
                peaks.append(s[i] - 1)
        for j in range(i+1, len(v)):
            if l[j] >= win // 2:
                peaks.append(s[j] - 1)
                i = j
        if j == len(v)-1:
            break

    return peaks

def find_valley_pivots(df, win):
    local_min = df.close.rolling(win).apply(lambda x: x.argmin()== win-1)
    local_min[:win] = 0
    
    v,s,l = find_runs(local_min)

    valleys = []
    i = 0
    while i < len(v):
        if l[i] >= win // 2:
            if s[i] > 0:
                valleys.append(s[i] - 1)
        for j in range(i+1, len(v)):
            if l[j] >= win // 2:
                valleys.append(s[j] - 1)
                i = j
        if j == len(v)-1:
            break

    return valleys


def trendline(df):
    peaks = find_peak_pivots(df, 20)
    valleys = find_valley_pivots(df, 20)

    p = np.polyfit(x=peaks, y = df.close[peaks].values, deg=1)
    upper_trendline = np.poly1d(p)(np.arange(0, len(df)))

    v = np.polyfit(x=valleys, y = df.close[valleys].values, deg=1)
    lower_trendline = np.poly1d(v)(np.arange(0, len(df)))

    candle = go.Candlestick(x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'],
                    line=dict({"width": 1}),
                    name="K 线",
                    increasing = {
                        "fillcolor": "rgba(255,255,255,0.9)",
                        "line": dict({"color": RED})
                    },
                    decreasing = dict(fillcolor =GREEN, line = dict(color =  GREEN))
                    )
    upper_trace = go.Scatter(x=df.index, y=upper_trendline, mode='lines', name='压力线')
    lower_trace = go.Scatter(x=df.index, y=lower_trendline, mode='lines', name='支撑线')
    fig = go.Figure(data=[candle,lower_trace, upper_trace ])

    fig.show()

## 形态检测

最后，我们对该标的在上涨之前的形态进行检测：

In [14]:
trendline(df.iloc[:179])

这个结果说明，旗形整理结束时，方向选择受大盘影响，仍有一定不确定性，但没有跌破前低，这是此后能凝聚共识、返身上涨的关键。

## 另一个示例

我们再来看一个最近一个月翻了7倍的标的：

In [15]:
code = "bj830799"
bars = ak.stock_zh_a_daily(symbol=code, adjust="qfq", start_date="20231101",end_date="20241022")

# 获取最近120个交易日的数据
df = bars.iloc[-200:]
df.index = np.arange(200)
df.tail()

candlestick(df)

这是未上涨前的形态：

In [16]:
candlestick(df.iloc[:187])

这是检测出来的旗形整理：

In [17]:
trendline(df.iloc[:187])